In [ ]:
# Trains class-balanced multinomial Logistic Regression models
# and generates explanatory plots for thesis (feature importance, confusion matrix, etc.)

import numpy as np
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Tuple, List
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, balanced_accuracy_score,
    classification_report, confusion_matrix
)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from joblib import dump

In [ ]:
# ======================
# CONFIG
# ======================
DATA_PATH = "/content/Data_To_Train_3_vitality.xlsx"
TEST_SIZE = 0.25
RANDOM_STATE = 42
DO_CV = False
CV_SPLITS = 3
CV_REPEATS = 5

# Band mapping per your sensor
BANDS = [f"B{i}" for i in range(1, 9)]
COASTAL, BLUE, GREEN1, GREEN, YELLOW, RED, RE, NIR = \
    "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8"
TARGETS = ["VitalityCategory", "StressCategory"]

In [ ]:
# ======================
# Utility Functions
# ======================
def load_table(path: str) -> pd.DataFrame:
    ext = Path(path).suffix.lower()
    if ext in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    return pd.read_csv(path)


def compute_features(df: pd.DataFrame) -> pd.DataFrame:
    eps = 1e-9
    X_raw = df[BANDS].apply(pd.to_numeric, errors="coerce").copy()
    X_idx = pd.DataFrame(index=df.index)

    # Normalized differences
    X_idx["NDVI"]   = (df[NIR] - df[RED])   / (df[NIR] + df[RED] + eps)
    X_idx["GNDVI"]  = (df[NIR] - df[GREEN]) / (df[NIR] + df[GREEN] + eps)
    X_idx["NDRE"]   = (df[NIR] - df[RE])    / (df[NIR] + df[RE] + eps)
    X_idx["NDWI"]   = (df[GREEN] - df[NIR]) / (df[GREEN] + df[NIR] + eps)

    # EVI
    X_idx["EVI"]    = 2.5*(df[NIR]-df[RED])/(df[NIR] + 6*df[RED] - 7.5*df[BLUE] + 1.0 + eps)

    # SAVI, MSAVI2, RDVI
    L = 0.5
    X_idx["SAVI"]   = (df[NIR]-df[RED])*(1+L)/(df[NIR]+df[RED]+L+eps)
    term = (2*df[NIR] + 1.0)
    X_idx["MSAVI2"] = 0.5*(term - np.sqrt(np.maximum(term**2 - 8*(df[NIR] - df[RED]), 0) + eps))
    X_idx["RDVI"]   = (df[NIR] - df[RED]) / np.sqrt(df[NIR] + df[RED] + eps)

    # Chlorophyll/structure & ratios
    X_idx["CIre"]      = (df[NIR] / (df[RE] + eps))   - 1.0
    X_idx["CIgreen"]   = (df[NIR] / (df[GREEN] + eps)) - 1.0
    X_idx["MTCI"]      = (df[NIR] - df[RE]) / (df[RE] - df[RED] + eps)
    X_idx["PRI"]       = (df[GREEN1] - df[GREEN]) / (df[GREEN1] + df[GREEN] + eps)
    X_idx["VARI"]      = (df[GREEN] - df[RED]) / (df[GREEN] + df[RED] - df[BLUE] + eps)
    X_idx["Ratio_RE_Red"]   = df[RE]  / (df[RED] + eps)
    X_idx["Ratio_Red_NIR"]  = df[RED] / (df[NIR] + eps)
    X_idx["Ratio_Blue_Red"] = df[BLUE]/ (df[RED] + eps)

    X_logs = pd.DataFrame(index=df.index)
    for b in BANDS:
        X_logs[f"log1p_{b}"] = np.log1p(df[b].astype(float).clip(lower=0))

    X = pd.concat([X_raw, X_idx, X_logs], axis=1)
    X = X.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how="any")
    return X


@dataclass
class ModelResult:
    target: str
    accuracy: float
    macro_f1: float
    bal_acc: float
    report: str
    conf_mat: np.ndarray
    classes: List[str]
    feature_names: List[str]
    coefs: np.ndarray


def make_model() -> Pipeline:
    """StandardScaler -> LogisticRegression (multinomial, class-balanced)"""
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(
            multi_class="multinomial",
            solver="saga",
            class_weight="balanced",
            C=1.0,
            max_iter=5000,
            random_state=RANDOM_STATE,
            n_jobs=-1
        ))
    ])
    return clf

In [ ]:
# ======================
# Visualization Helpers
# ======================
def plot_confusion_matrix(cm, classes, target_name, outdir):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues",
                xticklabels=classes, yticklabels=classes)
    plt.title(f"{target_name} - Confusion Matrix")
    plt.ylabel("True")
    plt.xlabel("Predicted")
    plt.tight_layout()
    plt.savefig(outdir / f"{target_name}_confusion_matrix.png", dpi=300)
    plt.close()


def plot_feature_importance(coefs, feature_names, target_name, outdir):
    # Use mean abs(coef) across classes for multinomial
    importance = np.mean(np.abs(coefs), axis=0)
    imp_df = pd.DataFrame({"Feature": feature_names, "Importance": importance})
    imp_df = imp_df.sort_values("Importance", ascending=False).head(20)

    plt.figure(figsize=(8, 6))
    sns.barplot(y="Feature", x="Importance", data=imp_df, palette="viridis")
    plt.title(f"{target_name} - Top 20 Feature Importances (|standardized coef|)")
    plt.tight_layout()
    plt.savefig(outdir / f"{target_name}_feature_importance.png", dpi=300)
    plt.close()


def plot_class_report(report_dict, target_name, outdir):
    metrics = pd.DataFrame(report_dict).T.drop(["accuracy"], errors="ignore")
    metrics = metrics.dropna(subset=["precision"], how="all")
    metrics = metrics.loc[~metrics.index.str.contains("avg", case=False)]

    plt.figure(figsize=(7, 5))
    metrics[["precision", "recall", "f1-score"]].plot(kind="bar")
    plt.title(f"{target_name} - Per-class Performance")
    plt.ylabel("Score")
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.savefig(outdir / f"{target_name}_class_report.png", dpi=300)
    plt.close()


In [ ]:
# ======================
# Main Train + Evaluate
# ======================
def fit_eval_one_target(X: pd.DataFrame, y_series: pd.Series, target_name: str):
    le = LabelEncoder()
    y = le.fit_transform(y_series.astype(str))
    class_names = list(le.classes_)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
    )

    model = make_model()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    mf1 = f1_score(y_test, y_pred, average="macro", zero_division=0)
    bacc = balanced_accuracy_score(y_test, y_pred)

    rep = classification_report(y_test, y_pred, target_names=class_names, output_dict=True, zero_division=0)
    cm  = confusion_matrix(y_test, y_pred)

    print(f"\n==== {target_name} (holdout) ====")
    print(f"Accuracy: {acc:.4f} | Balanced Acc: {bacc:.4f} | Macro-F1: {mf1:.4f}")

    coefs = model.named_steps["logreg"].coef_
    feature_names = X.columns.tolist()

    res = ModelResult(target_name, acc, mf1, bacc,
                      classification_report(y_test, y_pred, target_names=class_names, zero_division=0),
                      cm, class_names, feature_names, coefs)

    # === Visualization ===
    outdir = Path("logreg_outputs") / f"plots_{target_name}"
    outdir.mkdir(parents=True, exist_ok=True)

    plot_confusion_matrix(cm, class_names, target_name, outdir)
    plot_feature_importance(coefs, feature_names, target_name, outdir)
    plot_class_report(rep, target_name, outdir)

    return res, model, le

In [ ]:
def main():
    print("Loading data...")
    df = load_table(DATA_PATH)

    need = set(BANDS + TARGETS)
    miss = need - set(df.columns)
    if miss:
        raise ValueError(f"Missing required columns: {sorted(miss)}")

    df = df.dropna(subset=TARGETS).reset_index(drop=True)
    X_full = compute_features(df)
    df = df.loc[X_full.index].reset_index(drop=True)
    X_full = X_full.reset_index(drop=True)
    print(f"Total features built: {X_full.shape[1]}")

    results = {}

    for tgt in TARGETS:
        print(f"\nTraining {tgt} model...")
        res, model, le = fit_eval_one_target(X_full, df[tgt], tgt)
        results[tgt] = res

        dump(model, f"logreg_outputs/logreg_multinomial_{tgt}.joblib")
        dump(le, f"logreg_outputs/label_encoder_{tgt}.joblib")

    # Combined summary plot
    summary_df = pd.DataFrame([{
        "Target": tgt,
        "Accuracy": results[tgt].accuracy,
        "MacroF1": results[tgt].macro_f1,
        "BalancedAcc": results[tgt].bal_acc
    } for tgt in results])
    summary_df.to_csv("logreg_outputs/holdout_summary.csv", index=False)

    plt.figure(figsize=(6, 4))
    summary_df.set_index("Target")[["Accuracy", "MacroF1", "BalancedAcc"]].plot(kind="bar")
    plt.title("Model Performance Comparison")
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.savefig("logreg_outputs/model_comparison_summary.png", dpi=300)
    plt.close()

    print("\nAll outputs and plots saved in ./logreg_outputs/")

In [ ]:
if __name__ == "__main__":
    main()